# (Old) Implemented image manipulation ideas
### `apply_blur`
The function takes a NumPy array representing an image as input. It converts the array to a PIL Image, applies a Box Blur filter with a radius of 10 to the image using the filter() method, and then converts the modified PIL Image back to a NumPy array before returning the result.

In [ ]:
def apply_blur(image: np.ndarray):
    pilImage = Image.fromarray(image)
    pilImage = pilImage.filter(ImageFilter.BoxBlur(10))
    return np.array(pilImage)

### `create_rotate_image`

In [ ]:
def create_rotate_image(angle: int):
    def rotate_image(image: np.ndarray):
        pilImage = Image.fromarray(image)
        pilImage = pilImage.rotate(angle=angle)
        return np.array(pilImage)
    return rotate_image

### `flip_image_horizontally`
The function takes a NumPy array representing an image as input, converts it to a PIL Image, flips the image horizontally using the transpose() method with the specified transformation (FLIP_LEFT_RIGHT), and then converts the horizontally flipped PIL Image back to a NumPy array before returning the result.

In [ ]:
def flip_image_horizontally(image: np.ndarray):
    pilImage = Image.fromarray(image)
    pilImage = pilImage.transpose(Image.FLIP_LEFT_RIGHT)
    return np.array(pilImage)

### `change_to_grayscale`
The function accepts a NumPy array representing an image as input, transforms it into a PIL Image, converts the color image to grayscale using the convert() method with the 'L' mode, and then converts the resulting grayscale PIL Image back to a NumPy array before returning the processed image.

In [ ]:
def change_to_grayscale(image: np.ndarray):
    pilImage = Image.fromarray(image)
    pilImage = pilImage.convert('L')
    return np.array(pilImage)

### `apply_gauss_noise`
The function generates Gaussian noise with a mean of 50 and a standard deviation of 10 using NumPy's random.normal() function. This noise is created to match the shape of the input image. The generated noise is then added to the original image using the OpenCV add() function, resulting in an image with applied Gaussian noise.

In [ ]:
def apply_gauss_noise(image: np.ndarray):
    gauss = np.random.normal(50, 10, image.shape).astype('uint8')
    return cv2.add(image, gauss)

### `blur_edges`

In [ ]:
def blur_edges(image: np.ndarray):
    # edge detection
    grey_scale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    horizontalEdgeImage = convolve2d(grey_scale_image, np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]]), mode='same', boundary='symm')
    verticalEdgeImage = convolve2d(grey_scale_image, np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]), mode='same', boundary='symm')
    plt.matshow(horizontalEdgeImage, cmap='gray')
    plt.show()
    plt.matshow(verticalEdgeImage, cmap='gray')
    plt.show()
    edge_image = pow(pow(horizontalEdgeImage, 2) + pow(verticalEdgeImage, 2), 0.5)
    plt.matshow(edge_image, cmap='gray')
    plt.show()

    plt.matshow(np.abs(edge_image), cmap='gray')
    plt.show()

    print(edge_image.shape)
    edge_image_all_colors = np.tile(edge_image, (1, 3))

    print(edge_image_all_colors.shape)
    edge_image_all_colors_reshaped = edge_image_all_colors.reshape(edge_image.shape[0], edge_image.shape[1], 3)

    print(edge_image_all_colors_reshaped.shape)
    image_edge_blur = image + ((edge_image_all_colors_reshaped - image) * 0.2).astype('uint8')
    plt.matshow(image_edge_blur)
    plt.show()

    laplacian = cv2.Laplacian(grey_scale_image, cv2.CV_64F)
    plt.matshow(laplacian, cmap='gray')
    plt.show()

    plt.matshow(np.abs(laplacian), cmap='gray')
    plt.show()

    return image_edge_blur

### `create_add_black_squares`
The outer function, `create_add_black_squares`, takes two integer parameters, `square_size` and `number_of_squares`, and returns an inner function `add_black_squares`. When this inner function is called with a NumPy array representing an image, it iteratively adds black squares to the image. The size and number of squares are determined by the parameters provided during the creation of the outer function. The positions of the black squares are randomly generated within the image dimensions, and the pixel values within the specified square regions are set to 0 (black). The resulting image is then adjusted to ensure pixel values remain within the valid 0-255 range before being returned.

In [ ]:
def create_add_black_squares(square_size: int, number_of_squares: int):
    def add_black_squares(image: np.ndarray):
        for _ in range(number_of_squares):
            x = np.random.randint(0, image.shape[0] - square_size)
            y = np.random.randint(0, image.shape[1] - square_size)
            image[x:x + square_size, y:y + square_size] = 0  # 0 because squares are black

        # Ensure the image values stay within 0-255 range
        return np.clip(image, 0, 255).astype(np.uint8)

    return add_black_squares

### `apply_bilateral_filter`
The function applies a bilateral filter to the input image using the OpenCV `bilateralFilter` function. The bilateral filter is a non-linear, edge-preserving smoothing filter that considers both spatial and intensity differences between pixels. The parameters used for this filter are set to 9 for the diameter of the pixel neighborhood, and 75 for both the color and spatial sigma values. These parameters control the extent of filtering in terms of pixel proximity and color similarity. The resulting filtered image is then returned as the output of the function.

In [ ]:
def apply_bilateral_filter(image):
    # bilateral filter (explain params)
    filtered_image = cv2.bilateralFilter(image, 9, 75, 75)
    return filtered_image

# (New) Implemented image manipulation ideas

### `apply_blur`
This filter utilizes a Box Blur technique to evenly blur an image. By averaging the colors of pixels within a specified radius (controlled by the strength parameter), it effectively reduces image detail and smoothens transitions. The radius of blur determines the degree of softness, with a larger radius causing a more significant blurring effect. This filter can be selectively applied to a specific region of the image, such as the face, based on provided keypoints.

In [ ]:
def apply_blur(image: Image, keypoints, only_face=True, strength=10) -> Image:
    modified_image = image.filter(ImageFilter.BoxBlur(strength))
    if only_face:
        return swap_images_at_face_position(image, keypoints, modified_image)
    else:
        return modified_image

### `apply_dithering`
The dithering process in this filter reduces the color range of the image to a predefined palette, in this case, 16 colors. It achieves this through a quantization process that groups similar colors, followed by a conversion back to an RGB format. The outcome is an image characterized by distinct color blocks and a notable reduction in color gradation. This effect can be applied globally to the image or localized to a region determined by facial keypoints.

In [ ]:
def apply_dithering(image: Image, keypoints, only_face=True) -> Image:
    modified_image = image.quantize(colors=16).convert('RGB')
    if only_face:
        return swap_images_at_face_position(image, keypoints, modified_image)
    else:
        return modified_image

### `apply_max_filter`
This filter operates by scanning the image and replacing each pixel with the maximum pixel value in its neighborhood, defined by a specified filter size. As a result, brighter areas within the filter's radius become more prominent, while darker regions are subdued. This enhances the luminance contrast and can accentuate certain features of the image. The filter can be applied to the entire image or restricted to a region identified by facial keypoints.

In [ ]:
def apply_max_filter(image: Image, keypoints, only_face=True) -> Image:
    modified_image = image.filter(ImageFilter.MaxFilter(9))
    if only_face:
        return swap_images_at_face_position(image, keypoints, modified_image)
    else:
        return modified_image

### `apply_min_filter`
This filter is the inverse of the Max Filter. It scans the image and replaces each pixel with the minimum pixel value in its neighborhood. This process emphasizes darker areas and reduces the prominence of brighter ones. The Min Filter accentuates shadows and darker regions, providing a contrasting effect to the Max Filter. It can be used across the whole image or targeted to a specific area using facial keypoints.

In [ ]:
def apply_min_filter(image: Image, keypoints, only_face=True) -> Image:
    modified_image = image.filter(ImageFilter.MinFilter(9))
    if only_face:
        return swap_images_at_face_position(image, keypoints, modified_image)
    else:
        return modified_image

### `apply_closing`
The Closing Filter combines the effects of the Min and Max Filters in sequence. Initially, it applies the Min Filter, which reduces the prominence of smaller bright areas, followed by the Max Filter, which enhances the surrounding brighter regions. This sequence effectively 'closes' small gaps and dark spots, resulting in a smoother appearance in bright areas of the image. This filter can be applied universally or selectively based on keypoints.

In [ ]:
def apply_closing(image: Image, keypoints, only_face=True) -> Image:
    modified_image = apply_min_filter(apply_max_filter(image, keypoints, False), keypoints, False)
    if only_face:
        return swap_images_at_face_position(image, keypoints, modified_image)
    else:
        return modified_image

### `apply_opening`
The Opening Filter reverses the sequence of the Closing Filter. It starts with the Max Filter, which diminishes small dark regions, followed by the Min Filter, which then reduces the surrounding darker areas. This 'opening' effect is particularly noticeable in darker parts of the image, creating a sense of expansion in these regions. The filter can be applied to the entire image or localized to a specific area using keypoints.

In [ ]:
def apply_opening(image: Image, keypoints, only_face=True) -> Image:
    modified_image = apply_max_filter(apply_min_filter(image, keypoints, False), keypoints, False)
    if only_face:
        return swap_images_at_face_position(image, keypoints, modified_image)
    else:
        return modified_image

### `apply_color_shift`
This filter introduces a random shift in the color channels of the image. It independently alters the red, green, and blue channels by a random value within a specified range. The effect of this is a shift in the overall color balance of the image, producing a variety of color tones and hues. The degree of color shift is controlled by the max_shift_intensity parameter. The filter can modify the entire image or be confined to a particular region, such as the face, based on keypoints.

In [ ]:
def apply_color_shift(image: Image, keypoints, only_face=True, max_shift_intensity=25) -> Image:
    r_shift = random.randint(-max_shift_intensity, max_shift_intensity)
    g_shift = random.randint(-max_shift_intensity, max_shift_intensity)
    b_shift = random.randint(-max_shift_intensity, max_shift_intensity)
    r, g, b = image.split()
    r = r.point(lambda i: i + r_shift)
    g = g.point(lambda i: i + g_shift)
    b = b.point(lambda i: i + b_shift)
    modified_image = Image.merge('RGB', (r, g, b))
    if only_face:
        return swap_images_at_face_position(image, keypoints, modified_image)
    else:
        return modified_image

### `apply_pixelate`
This filter creates a pixelation effect by initially reducing the image's resolution and then scaling it back to its original size. The initial downscaling reduces detail, creating larger 'blocks' of color, and the subsequent upscaling maintains this blocky appearance. The degree of pixelation is dictated by the pixel_size parameter, with larger values producing more pronounced pixelation. The effect can be applied to the whole image or focused on a specific area, like the face, as determined by keypoints.

In [ ]:
def apply_pixelate(image: Image, keypoints, only_face=True, pixel_size=10) -> Image:
    small = image.resize((image.size[0] // pixel_size, image.size[1] // pixel_size), Image.NEAREST)
    modified_image = small.resize(image.size, Image.NEAREST)
    if only_face:
        return swap_images_at_face_position(image, keypoints, modified_image)
    else:
        return modified_image

### `apply_morph_eyes`
This filter specifically targets the eyes in an image to apply a morphing effect. It first checks for the presence of facial keypoints and, if found, identifies the positions of the left and right eyes. The filter then applies a morphing algorithm to these eye regions. The radius parameter, which is dynamically set based on the size of the detected face, defines the area around each eye that is affected by the morphing. The morph_strength parameter controls the intensity of the morphing effect. The result is a transformation of the eye regions, creating a unique, modified appearance of the eyes in the image.

In [ ]:
def apply_morph_mouth(image: Image, keypoints, radius=75, morph_strength=10.0) -> Image:
    if len(keypoints) == 0:
        return image
    image_cv = np.array(image)
    image_cv = cv2.cvtColor(image_cv, cv2.COLOR_RGB2BGR)
    mouth_points = [face_keypoints[key] for _, face_keypoints, _, _ in keypoints for key in
                    ['left_mouth', 'right_mouth', 'nose']]
    radius = keypoints[0][0][2] / 6
    image_cv = morph_image(image_cv, mouth_points, radius, morph_strength)
    return Image.fromarray(cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB))

### `apply_morph_mouth`
This filter is designed to morph the mouth area of an image. After ensuring that facial keypoints are present, it locates the positions of the left and right mouth corners and the nose. These points define the region around the mouth to be morphed. Similar to the Morph Eyes Filter, the radius for the morphing effect is determined based on the face size, specifically set to half the radius used for the eyes. The morph_strength parameter adjusts the level of morphing applied. This filter alters the mouth region, modifying its appearance in a distinctive way.

In [ ]:
def apply_morph_eyes(image: Image, keypoints, radius=75, morph_strength=10.0) -> Image:
    if len(keypoints) == 0:
        return image
    image_cv = np.array(image)
    image_cv = cv2.cvtColor(image_cv, cv2.COLOR_RGB2BGR)
    eye_points = [face_keypoints[key] for _, face_keypoints, _, _ in keypoints for key in ['left_eye', 'right_eye']]
    radius = keypoints[0][0][2] / 3
    image_cv = morph_image(image_cv, eye_points, radius, morph_strength)
    return Image.fromarray(cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB))

### `apply_morph_all`
This filter applies a comprehensive morphing effect to all key facial features. It utilizes all the detected facial keypoints, including points defining the face outline. The filter calculates a radius that is smaller compared to the previous filters, as it applies the morphing effect more broadly across the face. The morph_strength parameter still controls the intensity of the morphing. This all-encompassing approach results in a more dramatic transformation of the entire face, altering multiple features simultaneously for a significant visual change.

In [ ]:
def apply_morph_all(image: Image, keypoints, radius=75, morph_strength=10.0) -> Image:
    if len(keypoints) == 0:
        return image
    image_cv = np.array(image)
    image_cv = cv2.cvtColor(image_cv, cv2.COLOR_RGB2BGR)
    all_points = [point for _, face_keypoints, outline, _ in keypoints for key, point in face_keypoints.items()] + [pt for _, _, outline, _ in keypoints for pt in outline]
    radius = keypoints[0][0][2] / 12
    image_cv = morph_image(image_cv, all_points, radius, morph_strength)
    return Image.fromarray(cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB))

### `apply_sunglasses`
This filter overlays sunglasses onto the face in an image. It identifies the positions of the left and right eyes using facial keypoints and calculates the angle between them to rotate the sunglasses image accordingly. The distance between the eyes is used to dynamically scale the sunglasses' size, ensuring they fit the face proportionally. The sunglasses image is resized and rotated before being superimposed onto the face, creating the appearance of the subject wearing sunglasses.

In [ ]:
def apply_sunglasses(image: Image, keypoints, scale_factor: float = 2.5) -> Image:
    foreground = Image.open('filters/sunglasses.png')
    for (box, face_keypoints, face_shape_landmarks, _) in keypoints:
        left_eye = face_keypoints['left_eye']
        right_eye = face_keypoints['right_eye']
        dx = right_eye[0] - left_eye[0]
        dy = right_eye[1] - left_eye[1]
        angle_radians = math.atan2(-dy, dx)
        angle_degrees = math.degrees(angle_radians)
        eye_distance = math.dist((right_eye[0], right_eye[1]), (left_eye[0], left_eye[1]))
        foreground_width_to_height_ratio = foreground.size[0] / foreground.size[1]
        foreground = foreground.resize(size=(
            int(scale_factor * eye_distance), int(scale_factor * eye_distance / foreground_width_to_height_ratio)))
        rotated_overlay = foreground.rotate(angle_degrees, expand=True)
        left_part = (scale_factor - 1) / 2
        left_upper_sunglasses = (int(left_eye[0] - eye_distance * left_part),
                                 int(left_eye[1] - eye_distance * left_part / foreground_width_to_height_ratio))
        left_upper_paste = (left_upper_sunglasses[0], int(left_upper_sunglasses[1] - math.fabs(
            math.cos(math.radians(90 - angle_degrees)) * scale_factor * eye_distance)))
        image.paste(rotated_overlay, left_upper_paste, rotated_overlay)
    return image

### `apply_whole_face_mask`
This filter applies a face mask image over the entire face. It uses facial keypoints to determine the position and orientation of the face. The mask is resized to match the width of the face and rotated to align with the angle between the eyes. The mask is then placed over the face, covering it entirely, simulating the effect of wearing a full-face mask.

In [ ]:
def apply_whole_face_mask(image: Image, keypoints) -> Image:
    foreground = Image.open('filters/whole_face_mask.png').convert("RGBA")
    for (box, face_keypoints, face_shape_landmarks, _) in keypoints:
        left_eye = face_keypoints['left_eye']
        right_eye = face_keypoints['right_eye']
        dx = right_eye[0] - left_eye[0]
        dy = right_eye[1] - left_eye[1]
        angle_radians = math.atan2(-dy, dx)
        angle_degrees = math.degrees(angle_radians)
        face_width = box[2]
        foreground_width_to_height_ratio = foreground.size[0] / foreground.size[1]
        foreground = foreground.resize(size=(face_width, int(face_width / foreground_width_to_height_ratio)))
        rotated_overlay = foreground.rotate(angle_degrees, expand=True)
        left_upper_face_mask = (box[0], box[1])
        left_upper_paste = (left_upper_face_mask[0], int(left_upper_face_mask[1] - math.fabs(
            math.cos(math.radians(90 - angle_degrees)) * face_width)))
        image.paste(rotated_overlay, left_upper_paste, rotated_overlay)
    return image

### `apply_medicine_mask`
This filter overlays a medical-style mask image onto the face. It locates the position of the mouth and nose using facial keypoints and adjusts the size and rotation of the mask image to align with these features. The mask is placed to cover the lower half of the face, resembling the appearance of wearing a medical mask.

In [ ]:
def apply_medicine_mask(image: Image, keypoints) -> Image:
    foreground = Image.open('filters/medicine_mask.png').convert("RGBA")
    for (box, face_keypoints, face_shape_landmarks, _) in keypoints:
        left_mouth = face_keypoints['left_mouth']
        right_mouth = face_keypoints['right_mouth']
        dx = right_mouth[0] - left_mouth[0]
        dy = right_mouth[1] - left_mouth[1]
        angle_radians = math.atan2(-dy, dx)
        angle_degrees = math.degrees(angle_radians)
        face_width = box[2]
        foreground_width_to_height_ratio = foreground.size[0] / foreground.size[1]
        foreground = foreground.resize(size=(face_width, int(face_width / foreground_width_to_height_ratio)))
        rotated_overlay = foreground.rotate(angle_degrees, expand=True)
        left_upper_face_mask = (box[0], face_keypoints['nose'][1])
        left_upper_paste = (left_upper_face_mask[0], int(left_upper_face_mask[1] - math.fabs(
            math.cos(math.radians(90 - angle_degrees)) * face_width)))
        image.paste(rotated_overlay, left_upper_paste, rotated_overlay)
    return image

### `apply_cow_pattern`
This filter applies a cow pattern overlay to the facial area of the image. It calculates the bounding box for the face using facial keypoints and resizes the cow pattern to fit this area. The pattern is then applied over the face with a specified level of transparency (alpha_of_cow_pattern), creating a cow-patterned effect on the facial area.

In [ ]:
def apply_cow_pattern(image: Image, keypoints, alpha_of_cow_pattern: int = 85) -> Image:
    foreground = Image.open('../backend/filters/cow_pattern.png').convert("RGBA")
    foreground_parts = Image.new('RGBA', image.size)
    for (box, face_keypoints, face_shape_landmarks, _) in keypoints:
        (minX, maxX), (minY, maxY), (width, height) = calculate_face_shape_landmarks_box_positions(face_shape_landmarks)
        new_foreground_part = foreground.resize((width, height), resample=Image.LANCZOS)
        foreground_parts.paste(new_foreground_part, (minX, minY), new_foreground_part)
    foreground_parts.putalpha(alpha_of_cow_pattern)
    image = apply_filter_on_faces(image, keypoints, foreground_parts)
    return image

### `apply_salt_n_pepper`
This filter generates a 'salt and pepper' noise effect and applies it over the facial area. The noise is created by randomly assigning black and white pixels in equal proportions and then resizing this noise pattern to fit the face. The pattern is applied with a specified alpha value (alpha_of_salt_n_pepper), overlaying the face with this distinctive noise effect.

In [ ]:
def apply_salt_n_pepper(image: Image, keypoints, alpha_of_salt_n_pepper: int = 90) -> Image:
    foreground_parts = Image.new('RGBA', image.size)
    for (box, face_keypoints, face_shape_landmarks, _) in keypoints:
        (minX, maxX), (minY, maxY), (width, height) = calculate_face_shape_landmarks_box_positions(face_shape_landmarks)
        pixels = np.zeros(width * height, dtype=np.uint8)
        pixels[:width * height // 2] = 255  # Set first half to white (value 255)
        np.random.shuffle(pixels)
        rgb_box = np.stack((pixels, pixels, pixels), axis=-1)
        rgb_box_reshaped = np.reshape(rgb_box, (height, width, 3))
        rgb_box_image = Image.fromarray(rgb_box_reshaped)
        rgb_box_image.putalpha(255)
        foreground_parts.paste(rgb_box_image, (minX, minY), rgb_box_image)
    foreground_parts.putalpha(alpha_of_salt_n_pepper)
    image = apply_filter_on_faces(image, keypoints, foreground_parts)
    return image

### `apply_hide_with_masks`
This filter randomly places multiple small face mask images over the image. It ensures that these masks do not overlap with the facial areas identified by the keypoints. Each mask is resized according to specified dimensions (`face_mask_width` and `face_mask_height`) and applied with a certain level of transparency (`alpha_of_masks`). This creates a scattered mask effect across the image, avoiding the actual facial areas.

In [ ]:
def apply_hide_with_masks(img: Image, keypoints, number_of_masks: int = 40,
                          face_mask_width: int = 75, face_mask_height: int = 75,
                          alpha_of_masks: int = 45) -> Image:
    foreground = Image.open('../backend/filters/whole_face_mask.png').convert('RGBA')
    foreground_alpha = apply_alpha_to_transparent_image(foreground, alpha_of_masks)
    face_and_mask_coordinates = find_face_rectangles_mtcnn(keypoints)
    mask_cords = find_free_coordinates_outside_of_rectangles(img, number_of_masks, face_mask_width, face_mask_height,
                                                             face_and_mask_coordinates)
    for mask_coords in mask_cords:
        resized_foreground = foreground_alpha.resize((face_mask_width, face_mask_height), resample=Image.LANCZOS)
        img.paste(resized_foreground, (mask_coords[0], mask_coords[1]), resized_foreground)

    return img

### `apply_highlight_keypoints`
This filter visually highlights the facial keypoints and connections between them on the image. It draws circles around each keypoint and lines connecting them, using different colors for different facial features. This filter serves to visually emphasize the positions and relationships of facial features as identified by the keypoints.

In [ ]:
def apply_highlight_keypoints(image: Image, keypoints) -> Image:
    draw = ImageDraw.Draw(image)
    if len(keypoints) > 0:
        for keypoint_set in keypoints:
            for j in range(len(keypoint_set[2])):
                x, y = keypoint_set[2][j]
                if j < len(keypoint_set[2]) - 1:
                    next_x, next_y = keypoint_set[2][j + 1]
                    draw.line((x, y, next_x, next_y), fill='lightgreen', width=3)
                radius = 5
                draw.ellipse((x - radius, y - radius, x + radius, y + radius), fill='green', outline='lightgreen')
            for feature, coords in keypoint_set[1].items():
                x, y = coords
                radius = 10
                draw.ellipse((x - radius, y - radius, x + radius, y + radius), fill='red', outline='red')
    return image

### `apply_distance_transformation`
This filter applies a distance transformation technique to an image. It starts by converting the image to grayscale and then to a binary format based on a specified threshold (in this case, 128). Binary dilation and erosion operations are performed on the binary image to highlight regions of change. Dilation expands the white areas, while erosion shrinks them. The filter then compares the dilated and eroded images, highlighting the differences with white pixels. The result is an image that emphasizes the structural changes in the original image, providing a unique visual representation of distance transformation.

In [ ]:
def apply_distance_transformation(image: Image) -> Image:
    image_np = np.array(image.convert('L'))
    threshold = 128
    binary_image = (image_np > threshold).astype(np.uint8)
    dilated = binary_dilation(binary_image, iterations=5)
    eroded = binary_erosion(binary_image, iterations=5)
    morphed_image = np.where(dilated != eroded, 255, 0).astype(np.uint8)
    return Image.fromarray(morphed_image).convert('RGB')

### `apply_vertical_edge`
 This filter emphasizes vertical edges in an image using a specific kernel in a convolution process. The kernel used is designed to respond strongly to vertical lines or edges by subtracting the pixel value on the left from the pixel value on the right. This operation enhances vertical features while diminishing horizontal features. The filter is particularly effective in highlighting vertical structures or details in an image, making them more pronounced against the background.

In [ ]:
def apply_vertical_edge(image: Image) -> Image:
    return image.filter(ImageFilter.Kernel((3, 3), (-1, 0, 1, -2, 0, 2, -1, 0, 1), 1, 0))

### `apply_horizontal_edge`
Similar to the Vertical Edge Filter, this filter is designed to accentuate horizontal edges in an image. It uses a convolution kernel that contrasts pixel values above and below each pixel in the image. By doing so, horizontal lines or edges become more pronounced. This filter is useful for emphasizing horizontal features or details within the image, enhancing their visibility and distinction from the rest of the image elements.

In [ ]:
def apply_horizontal_edge(image: Image) -> Image:
    return image.filter(ImageFilter.Kernel((3, 3), (-1, -2, -1, 0, 0, 0, 1, 2, 1), 1, 0))